# MLIP 2023: psiflow tutorial
**author**: Sander Vandenhaute

copyright (c) 2023 Ghent University. This tutorial is licensed under the MIT license, and is part of the official psiflow codebase.

---

This tutorial consists of the following parts:

- **Parsl 101**: Apps and Futures
- **Building blocks**: Dataset, Model, Walker, Reference
- **Online learning**: MACE and umbrella sampling
- **Inference**: free energy profile via MBAR

In [ ]:
%%capture

# set up container engine
!sudo add-apt-repository -y ppa:apptainer/ppa
!sudo apt update
!sudo apt install -y apptainer

# download and install psiflow + dependencies
!pip install 'psiflow[parsl] @ git+https://github.com/molmod/psiflow' py3Dmol pymbar

# get notebook source files and helper functions
!wget https://raw.githubusercontent.com/molmod/psiflow/main/examples/notebook/notebook_utils.py
!wget https://raw.githubusercontent.com/molmod/psiflow/main/examples/notebook/zerokelvin.xyz

# colab stuff
import site
site.main()
import os
os.environ["WANDB_ANONYMOUS"] = "must"

In [ ]:
import os
import shutil
from pathlib import Path
import numpy as np

from parsl.app.app import python_app

import psiflow
from psiflow.data import Dataset
from psiflow.models import MACEModel, MACEConfig, load_model
from psiflow.walkers import DynamicWalker, PlumedBias, BiasedDynamicWalker
from psiflow.reference import PySCFReference
from psiflow.learning import SequentialLearning
from psiflow.metrics import Metrics

Before we begin, we **highly** recommend you to use Weights & Biases for monitoring online learning workflows with psiflow. Head over to [wandb.ai](https://wandb.ai), login via GitHub/Google/... , go to [wandb.ai/authorize](https://wandb.ai/authorize), and copy paste the API key in the cell below to set it as an environment variable:

In [ ]:
os.environ['WANDB_API_KEY'] = your_api_key_from_wandb

Psiflow can be configured for execution on arbitrary types of compute resources, including local workstations, SLURM clusters, Google Cloud instances, Kubernetes, HTCondor, etc. This tutorial will use the Mahti SLURM cluster provided by CSC, and a `mahti.yaml` configuration file is available in `/scratch/project_2008666/psiflow`. Copy it into the working directory of this notebook and load it such that psiflow can use its resources:

In [ ]:
shutil.rmtree('psiflow_internal', ignore_errors=True) # auto-remove temporary cache dir if it exists
psiflow.load_from_yaml('mahti.yaml')

Lastly, it might be handy to keep the following tabs open:

- the [repository](https://github.com/molmod/psiflow) and [documentation](https://molmod.github.io/psiflow) of psiflow
- a login shell on Mahti/Puhti, to be able to track job submissions and inspect logs (via `ssh your_username@mahti.csc.fi`)


# Parsl 101: Apps and Futures
Psiflow is built on Parsl, a parallel execution library for Python. Its core concepts are **Apps** and **Futures**. In their simplest
form, apps are just functions, and futures are the result of an app given
a set of inputs. The key thing to understand is that a Future already exists *before* the function evaluation has completed. In essence, a Future _promises_ that, at some time in the future, it will
contain the actual result of a function evaluation. Take a look at the following
example:

In [ ]:
def sum_integers(a, b):
    return a + b

sum_integers_app = python_app(sum_integers, executors=['default_threads'])  # ignore the executors=... part for now

We've defined a simple function which accepts two integers and returns their sum; `sum_integers`. Next, we've created a Parsl `App` from this function. Both of them essentially represent the same thing (the addition of two floats), but the `App` supports _delayed_ execution using the Python built-in `concurrent` library. It will not return the result of the addition itself, but it will return a `concurrent.Future`. You can think of it as a kind of wrapper which will contain the actual result of the function _when it becomes available_. Let us compare both:

In [ ]:
sum_integers(3, 4)

In [ ]:
future = sum_integers_app(3, 4) # generate a Future which represents the result of sum_integers(3, 4)
future

To actually force Python to wait until a `Future` has completed and obtain the actual value, we use `.result()`:

In [ ]:
future.result()

Apps can be configured to run on any of the execution resources which were configured in the YAML configuration file. In the above case, we asked Parsl to execute the `sum_integers_app` via local threads (i.e. `default_threads`) on the interactive job in which this notebook is running. We can also ask it to run on any of the compute nodes of Mahti as configured in `mahti.yaml`:

In [ ]:
sum_integers_app = python_app(sum_integers, executors=['ModelEvaluation'])  # or ModelTraining, ReferenceEvaluation; see psiflow config
future = sum_integers_app(4, 5)

Open a login shell on Mahti and check your queued and running jobs with `squeue -u <your_username>`. Normally, you should observe that Parsl has submitted a job as configured in the `ModelEvaluation` section of the configuration. Parsl will wait until it is started, and execute the function on the allocated compute node. The result is automatically sent back to this notebook via TCP.

In [ ]:
future.result()

That's more or less all you need to know about Parsl for the purposes of this tutorial! While psiflow is built on Parsl, users will never actually have to use `python_app`, and only occasionally use `.result()` on a `Future`. For most purposes, you can use psiflow like any other library.
For a more in-depth look at Parsl, check out the [repository](https://github.com/parsl/parsl) and/or the [documentation](https://parsl.readthedocs.io/en/stable/).

# Building blocks

Unless otherwise mentioned, psiflow uses [ASE's unit system](https://wiki.fysik.dtu.dk/ase/ase/units.html).

## Atomic data
In psiflow, a set of atomic configurations is represented using the `Dataset` class.
It may represent training/validation data for model development, or
a trajectory of snapshots that was generated using molecular dynamics.
A `Dataset` instance mimics the behavior of a list of ASE `Atoms` instances:

In [ ]:
data  = Dataset.load('data/train.xyz')    # create a psiflow Dataset from a file; contains QM energy/forces
data_subset = data[:10]                   # create a new Dataset instance with the first 10 configurations
data  = data_subset + data[10:]           # combining two datasets is easy

The main difference between a psiflow `Dataset` instance and an actual Python `list` of
`Atoms` is that a `Dataset` can represent data __that will be generated in the future__.
Similar to the `sum_integers` example above, a `Dataset` object can already be created even though
the underlying atomic configurations (e.g. snapshots from a molecular dynamics run) are not yet available.

In [ ]:
data[4]

In [ ]:
data[4].result()

In general, you can make psiflow wait for any uncompleted futures using the following command:

In [ ]:
psiflow.wait()

## Trainable potentials
Once we know how datasets are represented, we can start defining models.
Psiflow defines an abstract `BaseModel` interface which each
particular ML potential should subclass (e.g. NequIP or MACE).
In addition, psiflow provides configuration dataclasses for each model with
reasonable defaults for each model's hyperparameters.

- __NequIP__    : implemented by `NequIPModel` and `NequIPConfig`
- __Allegro__   : implemented by `AllegroModel` and `AllegroConfig`
- __MACE__      : implemented by `MACEModel` and `MACEConfig`

Each model is required to implement the following methods

- `initialize`: compute energy shifts and scalings as well as the average number
of neighbors using a given training dataset and initialize model weights.
- `train`: train the parameters of a model using two separate datasets, one for training and one for validation. The current model parameters are used as starting parameters for the training
- `evaluate`: compute the energy, force, and stress predictions over a dataset of configurations

The following example illustrates how `Dataset` and model instances can be
used to train and evaluate errors.

In [ ]:
config = MACEConfig()
config.num_channels = 16      # modify MACE parameters to sane defaults
config.max_L = 2
config.r_max = 5.5
config.patience = 5
config.energy_weight = 100
model = MACEModel(config)     # create model instance

In [ ]:
train, valid = data[:100].shuffle().split(0.9)   # create randomized 90/10 training/validation split of first 100 states
model.initialize(train)                    # this will calculate the scale/shifts, and average number of neighbors
model.train(train, valid)                  # train using supplied datasets

_ = model.save('./my_first_training/')     # save trained model!

Notice how the above cell instantly 'completes'! This is because the initialization/train/save methods are all implemented as apps which are executed on other compute nodes (e.g. `model.train()` will be dispatched to the Mahti partition specified under 'ModelTraining' in the `mahti.yaml` configuration). Unless explicitly told to do so, the current notebook will not wait for the training run to be finished. In Parsl terms: the current notebook will not wait for the training `Future` to complete.

Let's evaluate the validation error of the trained model:

In [ ]:
valid_model = model.evaluate(valid)   # create Dataset with predicted energy/forces
errors = Dataset.get_errors(
    valid,                            # contains QM labels
    valid_model,                      # contains model predictions
    properties=['energy', 'forces'],  # which properties to compute error for
)

Because the model is internally represented using a `Future`, any downstream inference tasks such as this one are also necessarily `Futures`. In this specific case, `errors` is a `Future` of a `numpy` array in which the energy and force errors per configuration are stored. If we request the result of this future, we will have to wait until model training actually completes. This should only take a few minutes:

In [ ]:
np.mean(errors.result(), axis=0)    # (energy, force) RMSEs in (meV/atom, meV/angstrom)

## Molecular simulation
Having trained a model, it is possible to explore the phase space
of the physical system in order to generate new geometries.
Phase space exploration in psiflow is done using **walkers**. In general, walkers start from a specific geometry
and use a model to generate a trajectory in phase space according to some ensemble.
For example, the `DynamicWalker` implements molecular dynamics with optional stochastic
Langevin control of temperature and/or pressure, the `BiasedDynamicWalker` additionally allows the use of PLUMED biasing during the simulation, and the `OptimizationWalker` performs energy minimizations. Let us consider the basic `DynamicWalker` as an example:

In [ ]:
walker = DynamicWalker(
        train[0],           # initialize walker to first configuration of training set
        timestep=0.5,       # Verlet timestep in fs
        steps=1000,         # number of timesteps to perform
        step=100,           # frequency with which states are sampled
        start=0,            # timestep at which sampling is started
        temperature=300,    # temperature, in kelvin
        pressure=None,      # pressure, in MPa. If None, then simulation is NVT
        seed=0,             # numpy random seed with which initial Boltzmann velocities are set
        )

metadata, trajectory = walker.propagate(model=model, keep_trajectory=True)

The `propagate` method returns two things:

- **metadata**: this contains the final state in which the walker arrived, as well as some additional information. Metadata is a Python `namedtuple`.
- **trajectory** (optional): when requested, the trajectory of the walker from initial to final state is returned as a `Dataset` instance. The default behavior (`keep_trajectory=False`) does **NOT** save the trajectory of the simulation to save storage/inode space because it is typically not used in online learning anyway.

In [ ]:
metadata.state.result()    # Atoms instance
metadata.counter.result()  # number of timesteps which have been performed
metadata.stdout            # filepath of the output log
metadata.time.result()     # total runtime of the simulation

In [ ]:
trajectory.length().result()    # 1000 // 100 + 1

## QM evaluation
Once we know how to train models and use them to generate new configurations, we still need one more component to 'close' the active learning loop: QM evaluation.
In psiflow, a specific QM level of theory (i.e. a specific quantum chemistry package along with all of its settings, basis sets, pseudopotentials etc) is defined using the `BaseReference` interface; CP2K calculations can be performed using the `CP2KReference`, whereas PySCF calculations can be performed using the `PySCFReference`.

In [ ]:
routine = """
from pyscf import dft

mf = dft.RKS(molecule)
mf.xc = 'pbe,pbe'

energy = mf.kernel()
forces = -mf.nuc_grad_method().kernel()
"""
basis = 'cc-pvtz'
spin = 0
reference = PySCFReference(routine, basis, spin)

A `reference` instance is almost exclusively used to evaluate a dataset of structures and insert the obtained energy
and forces in the `info` and `arrays` attributes of the constituting states.

In [ ]:
labeled = reference.evaluate(trajectory)

Importantly, QM evaluations of a dataset of configurations will proceed in a massively parallel manner. That is, Parsl will request as many nodes as possible (as determined by the `max_blocks` keyword in `mahti.yaml`) and maximally distribute each of the singlepoint evaluations. This is possible because each evaluation is entirely independent from every other evaluation.
Let us evaluate the obtained trajectory with 11 states, and observe the SLURM queue in a login shell using `squeue -u <your_username>`:

In [ ]:
Dataset.get_errors(
    labeled,
    model.evaluate(trajectory),
    properties=['energy', 'forces'],
).result().mean(axis=0)

# Online learning: MACE and umbrella sampling

## The reaction
Our application is inspired by [a recent JCTC paper](https://pubs.acs.org/doi/full/10.1021/acs.jctc.2c00874) in which elementary proton transfer reactions are investigated using hybrid DFT. To keep things computationally tractable, we'll stick to the GGA level of theory (PBE-D3) and classical molecular dynamics (i.e. without taking nuclear quantum effects into account).
The reaction and a corresponding suitable collective variable are shown below:

<div>
<img src="data/reaction.png" width="500"/>
<figcaption>Figure 1: Overview of the proton exchange and definition of the reaction coordinate.</figcaption>
</div>

You'll find a short zero-kelvin trajectory of the transition in the 'data' folder; run the cell below to visualize it.
If you a priori do not know the reaction path, psiflow can use moving harmonic restraints to automatically 'discover' it during online learning. For more on that, see the `IncrementalLearning` class and its associated examples in the [psiflow repository](https://github.com/molmod/psiflow).

In [ ]:
zerokelvin = Dataset.load('data/zerokelvin.xyz')
notebook_utils.visualize_trajectory(zerokelvin.as_list().result())

Since this is a rare event with a nontrivial free energy barrier, online learning based on pure equilibrium molecular dynamics is not or barely going to sample the transition state in which protons are located at the center of both molecules. This is rather problematic since it would imply that the transition region is not going to be represented in our training data, and our final model will perform poorly.
To circumvent this, we will integrate umbrella sampling in our online learning workflow.

Let us initialize a [PLUMED](https://www.plumed.org/) bias object with a CV as shown above.

In [ ]:
# standard PLUMED input; see docs for details
plumed_input = """
UNITS LENGTH=A ENERGY=kj/mol TIME=fs

C1: COORDINATION GROUPA=1 GROUPB=10 R_0=1.4
C2: COORDINATION GROUPA=8 GROUPB=10 R_0=1.4
C3: COORDINATION GROUPA=3 GROUPB=5 R_0=1.4
C4: COORDINATION GROUPA=6 GROUPB=5 R_0=1.4

CV: COMBINE ARG=C1,C2,C3,C4 COEFFICIENTS=1.0,-1.0,-1.0,1.0 PERIODIC=NO
RESTRAINT ARG=CV AT=0 KAPPA=3000
"""
bias = PlumedBias(plumed_input)

Umbrella sampling is essentially a superposition of multiple molecular dynamics simulations, each with a harmonic bias at a particular CV value.
Psiflow has a convenient constructor to initialize such a set of walkers:

In [ ]:
walkers = BiasedDynamicWalker.distribute(
    50,
    data_start=zerokelvin,  # initial configurations for walkers; the most suited one is taken for each umbrella
    bias=bias,
    timestep=0.5,
    steps=1000,
    step=50,
    max_excess_temperature=500, # reset walker if temperature explodes
    variable='CV',
    min_value=-1.4,
    max_value=1.4,
)

# check whether umbrellas are properly distributed
for walker in walkers:
    restraint = walker.bias.get_restraint('CV')
    print('kappa: {} kJ/mol\t\tcenter: {}'.format(*restraint))

Let's also improve the hyperparameters of our model (for more information, see [MACE](https://github.com/acesuit/mace)):

In [ ]:
config.patience = 20
config.energy_weight = 50
config.lr = 0.0004
config.batch_size = 64
model = MACEModel(config)

# add atomic energies to model--> better generalizability
model.add_atomic_energy('H', reference.compute_atomic_energy('H'))
model.add_atomic_energy('O', reference.compute_atomic_energy('O'))
model.add_atomic_energy('C', reference.compute_atomic_energy('C'))

Finally, let us define our online learning workflow and hyperparameters.
In psiflow, the `BaseLearning` class defines an interface for any kind of learning algorithm that involves a list of walkers, a QM reference object, and a trainable model.
In this example, we'll use the `SequentialLearning` class, which does the basic active learning loop (sampling, QM evaluation, training) without too many bells and whistles regarding data selection or walker steering.
Let's discuss the most relevant hyperparameters:

- `path_output`: location where output from each iteration can be saved
- `niterations`: the number of active learning iterations to perform
- `train_from_scratch`: whether train from scratch in each iteration, or continue training from the last iteration
- `metrics`: tracks the learning and saves metrics to files and, optionally but strongly recommended, to Weights & Biases
- `error_thresholds_for_reset`: when a sampled state is QM evaluated and the model error prior to training is higher than this, the walker is reset to its initial configuration. This is important in order to guarantee a physical sampling of structures, especially in the first few iterations.
- `temperature_ramp`: initial and final temperature of the walkers, along with the number of iterations over which the temperature should be increased. Increments are performed linearly in beta, i.e. inverse temperature.


In [ ]:
learning = SequentialLearning(
    path_output='learning_output',
    niterations=10,
    train_valid_split=0.9,
    train_from_scratch=True,
    pretraining_nstates=100,
    metrics=Metrics('umbrella_learning', 'formic_acid'),
    error_thresholds_for_reset=(10, 200), # in meV/atom, meV/angstrom
    temperature_ramp=(100, 600, 3),       # increase the temperature from 100 K to 600 K in three iterations; stick to 600 K afterwards
    )
data = learning.run(model, reference, walkers)

If all goes well, this should take some time to finish. You can track the progress at [wandb.ai](https://wandb.ai).
You can also use a login shell to investigate the `learning_output` and `psiflow_internal/000/task_logs/*` directories in a little more detail. They contain output files related to QM evaluation, MD, or training, as well as intermediate models, datasets, and walker states.

If you don't want to wait, we'd recommend you to explore the output of a fully completed run [here](https://wandb.ai/svandenhaute/formic_acid?workspace=user-svandenhaute). It contains output from a sequential learning run (in which we start from our zero-kelvin transition path and use umbrella sampling) as well as an incremental learning run in which psiflow discovers the reaction path itself using moving harmonic restraints during online learning.

# Inference: free energy profile via MBAR

The building blocks of psiflow (walker, model, reference) are as suited for training as they are for model inference, i.e. the estimation of physical observables! Just like online learning, computing physical observables pretty much always requires some kind of ensemble average which in turn relies on phase space exploration. In other words, we can just use the walkers as regular MD engines, and use the resulting trajectories in order to extract properties.

In this particular case, we will estimate the free energy profile of the proton transfer along the proposed collective variable. This will require a similar ensemble of walkers as during our online learning, except that the sampling times need to be slightly longer:

In [ ]:
walkers = BiasedDynamicWalker.distribute(
    50,
    data_start=zerokelvin,
    bias=bias,
    timestep=0.5,
    steps=50000,
    step=100,
    temperature=300,
    variable='CV',
    min_value=-1.4,
    max_value=1.4,
)
model = load_model('data/learning_output/8')        # load a fully trained model because the online learning most likely took too long

The following lines of code use the walkers to generate 25 ps trajectories, evaluate the CV over those trajectories, and save them to separate files. **Note that while we iterate over the walkers in a sequential for loop, the actual simulations are still performed in a massively parallel manner because Parsl recognizes they are completely independent**. This is possible because there are no `.result()` calls in the loop, so the walkers are actually only just setting up the `Futures` of the simulations -- the rest happens in the back.

In [ ]:
CV_arrays = []
for walker in walkers:
    _, trajectory = walker.propagate(model=model, keep_trajectory=True)  # trajectory is of type Dataset
    CV_arrays.append(bias.evaluate(trajectory, variable='CV'))           # these are all Futures!

It's only when we actually want to save the CV values that we'll require each of the Futures to complete (which in turn will require the molecular dynamics simulation to complete).

In [ ]:
path_output = Path.cwd() / 'umbrella_output'
path_output.mkdir(exist_ok=True)
for i, array in enumerate(CV_arrays):
    np.savetxt(path_output / '{}.txt'.format(i), array.result())

Finally, let's process and plot our obtained free energies! For this, we'll use the `pymbar` package (which has been pip-installed in the current environment).

In [ ]:
x, y = notebook_utils.generate_fes(
    Path.cwd() / 'data' / 'umbrella_output',
    centers=np.linspace(-1.4, 1.4, 50),
    kappa=3000,
    temperature=300,
    nbins=20,
)